# Cleaned London Count and Spatial Data - Analysis

## Rationale & Methodology

### Descriptive Analytics
- Import the necessary libraries
- Group / format data as necessary
- Discover which variables impact the total cyclist counts via heatmaps
- Fit to a multilinear regression model (with caveats, eg check for multicollinearity, breushpagan etc)
- For demographic / gender data: utilise t-tests to check if the percentage of men and women are significant year on year
- Utilise Tableau and maybe SQL to get top 10 sites etc.
- Utilise faceted plots to find as many sub-trends as possible.

### Diagnostic Analytics
- Match patterns seen in the data with research done by Javier.

### Predictive Analytics
- Fit the data into an MLR model or use ARIMA / time-series decomposition to break down the charts


In [ ]:
# Import the necessary libraries and packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install plotly
import plotly.express as px

In [ ]:
# Load the cleaned London dataset.
london_clean = pd.read_csv("london_count_and_site_Saurav_161022.csv")

In [ ]:
# Check the cleaned dataset.
london_clean.head()

From the data we can tell the following unique values
- reference: https://www.askpython.com/python/built-in-methods/unique-values-from-a-dataframe

In [ ]:
# Find all the column names.
list(london_clean)

In [ ]:
# Remove unnecessary column names.
london_clean.pop("Unnamed: 0")

# View the list of columns
list(london_clean)

In [ ]:
# Find the unique entries of independent variables of interest.
print(pd.unique(london_clean.weather))
print(pd.unique(london_clean.survey_year))
print(pd.unique(london_clean.period))
print(pd.unique(london_clean.direction))
print(pd.unique(london_clean.day_of_week))
print(pd.unique(london_clean.season))
print(pd.unique(london_clean.surveydescription))
print(pd.unique(london_clean.borough))
print(pd.unique(london_clean.functional_cycling_area))
print(pd.unique(london_clean.month))

In [ ]:
# Possible set of independent variables for investigation: 
# Weather, Period, Direction, Day of the week, Season, Functional cycling Area

# Borough Names can be used for labels.

# Utilise Macro-level year-on-year analysis first
# Then drill down month-on-month

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_weather = london_clean.groupby(["survey_year", "site_id", "weather"]).agg("sum").reset_index()
london_weather = london_weather[["survey_year","site_id","weather","total_cycles"]]

london_weather_YoY_data = pd.merge(london_weather,london_locations,how="left",on="site_id")

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html
# https://www.geeksforgeeks.org/how-to-fill-nan-values-with-mean-in-pandas/

# Heatmaps of total cyclist count by Weather condition

# Good Weather

In [ ]:
fig = px.density_mapbox(london_weather_YoY_data[london_weather_YoY_data["weather"]=="Good"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "weather":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london in Good Weather",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

## Damp weather

In [ ]:
fig = px.density_mapbox(london_weather_YoY_data[london_weather_YoY_data["weather"]=="Damp"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "weather":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london in Damp Weather",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

## Rainy weather

In [ ]:
fig = px.density_mapbox(london_weather_YoY_data[london_weather_YoY_data["weather"]=="Rain"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "weather":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london in Rainy Weather",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

## Dangerous Weather

In [ ]:
fig = px.density_mapbox(london_weather_YoY_data[london_weather_YoY_data["weather"]=="Dangerous_Conditions"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "weather":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london in Dangerous Weather",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Heatmaps of total cyclist count by time period

## Early Morning (6-7am)

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_period = london_clean.groupby(["survey_year", "site_id", "period"]).agg("sum").reset_index()
london_period = london_period[["survey_year","site_id","period","total_cycles"]]

london_period_YoY_data = pd.merge(london_period,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_period_YoY_data[london_period_YoY_data["period"]=="Early Morning (06:00-07:00)"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                   lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "period":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london in the Early Morning",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

## Morning Peak Hours

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_period = london_clean.groupby(["survey_year", "site_id", "period"]).agg("sum").reset_index()
london_period = london_period[["survey_year","site_id","period","total_cycles"]]

london_period_YoY_data = pd.merge(london_period,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_period_YoY_data[london_period_YoY_data["period"]=="AM peak (07:00-10:00)"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "period":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london during Morning Peak hours",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

## Inter-peak Hours (10am-4pm)

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_period = london_clean.groupby(["survey_year", "site_id", "period"]).agg("sum").reset_index()
london_period = london_period[["survey_year","site_id","period","total_cycles"]]

london_period_YoY_data = pd.merge(london_period,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_period_YoY_data[london_period_YoY_data["period"]=="Inter-peak (10:00-16:00)"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "period":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london during Inter-Peak hours",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

## Afternoon Peak hours (4-7pm)

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_period = london_clean.groupby(["survey_year", "site_id", "period"]).agg("sum").reset_index()
london_period = london_period[["survey_year","site_id","period","total_cycles"]]

london_period_YoY_data = pd.merge(london_period,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_period_YoY_data[london_period_YoY_data["period"]=="PM peak (16:00-19:00)"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "period":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london during Afternoon Peak hours",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

## Evening hours (7pm-10pm)

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_period = london_clean.groupby(["survey_year", "site_id", "period"]).agg("sum").reset_index()
london_period = london_period[["survey_year","site_id","period","total_cycles"]]

london_period_YoY_data = pd.merge(london_period,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_period_YoY_data[london_period_YoY_data["period"]=="Evening (19:00-22:00)"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "period":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london during Evening hours",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Heatmap showing the average number of cyclists throughout the day

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_period2 = london_clean.groupby(["site_id", "period","time"]).agg("sum").reset_index()
london_period2 = london_period2.sort_values(by="time",ascending=True)
london_period2 = london_period2[["site_id","period","total_cycles"]]

london_period2_data = pd.merge(london_period2,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_period2_data,
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="period",
                        zoom=0,
                        hover_name="location",
                        hover_data={"site_id":False,
                                    "period":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Average number of cyclists throughout different times of the day",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# make sure you're counting the right thing
# read tools functool
# give me the best youve got tmr etc

# Heatmap showing the total number of cyclists by day of the week

## Monday

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_day_of_wk = london_clean.groupby(["survey_year", "site_id", "day_of_week"]).agg("sum").reset_index()
london_day_of_wk = london_day_of_wk[["survey_year","site_id","day_of_week","total_cycles"]]

london_DOW_data = pd.merge(london_day_of_wk,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_DOW_data[london_DOW_data["day_of_week"]=="Monday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "day_of_week":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london on Monday",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Tuesday

In [ ]:
fig = px.density_mapbox(london_DOW_data[london_DOW_data["day_of_week"]=="Tuesday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "day_of_week":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london on Tuesday",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Wednesday

In [ ]:
fig = px.density_mapbox(london_DOW_data[london_DOW_data["day_of_week"]=="Wednesday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "day_of_week":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london on Wednesday",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Thursday

In [ ]:
fig = px.density_mapbox(london_DOW_data[london_DOW_data["day_of_week"]=="Thursday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "day_of_week":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london on Thursday",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Friday

In [ ]:
fig = px.density_mapbox(london_DOW_data[london_DOW_data["day_of_week"]=="Friday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "day_of_week":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london on Friday",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Saturday 

In [ ]:
fig = px.density_mapbox(london_DOW_data[london_DOW_data["day_of_week"]=="Saturday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "day_of_week":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london on Saturday",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Sunday

In [ ]:
fig = px.density_mapbox(london_DOW_data[london_DOW_data["day_of_week"]=="Sunday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "day_of_week":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london on Sunday",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Heatmaps showing year-on-year total cyclist counts in London by season

# Winter

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_season = london_clean.groupby(["survey_year", "site_id", "season"]).agg("sum").reset_index()
london_season = london_season[["survey_year","site_id","season","total_cycles"]]

london_season_data = pd.merge(london_season,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_season_data[london_season_data["season"]=="Winter"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "season":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london in Winter",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Spring

In [ ]:
fig = px.density_mapbox(london_season_data[london_season_data["season"]=="Spring"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "season":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london in Spring",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Autumn

In [ ]:
fig = px.density_mapbox(london_season_data[london_season_data["season"]=="Autumn"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "season":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london in Autumn",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Summer

In [ ]:
fig = px.density_mapbox(london_season_data[london_season_data["season"]=="Summer"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "season":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in london in Summer",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Total Cyclist count by Functional Cycling Area

## Inner

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_areas = london_clean.groupby(["survey_year", "site_id", "functional_cycling_area"]).agg("sum").reset_index()
london_areas = london_areas[["survey_year","site_id","functional_cycling_area","total_cycles"]]

london_area_data = pd.merge(london_areas,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_area_data[london_area_data["functional_cycling_area"]=="Inner"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "functional_cycling_area":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in Inner london",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Outer London

In [ ]:
fig = px.density_mapbox(london_area_data[london_area_data["functional_cycling_area"]=="Outer"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "functional_cycling_area":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in Outer london",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Central London

In [ ]:
fig = px.density_mapbox(london_area_data[london_area_data["functional_cycling_area"]=="Central"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="survey_year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"survey_year":True,
                                    "site_id":False,
                                    "functional_cycling_area":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year variation in total Cyclist Counts in Central London",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Facetplots

In [ ]:
list(london_clean)

In [ ]:
london_clean_cycling_area = london_clean[["functional_cycling_area","borough"]]

london_grpby_bor_weather = london_clean.groupby(["survey_year","borough","period"]).agg("sum").reset_index()
london_grpby_bor_weather = london_grpby_bor_weather[["survey_year",
                                                    "borough",
                                                    "period",
                                                    "total_cycles"]]
london_grpby_bor_weather_mrg = pd.merge(london_grpby_bor_weather,london_clean_cycling_area,on="borough",how="left")

london_clean_catplot_mar = sns.catplot(
                            x="survey_year",
                            y= "total_cycles",
                            row= "borough",
                            col= "period",
                            kind="point",
                            hue="functional_cycling_area",
                            data=london_grpby_bor_weather_mrg) 

In [ ]:
london_clean_cycling_area = london_clean[["functional_cycling_area","borough"]]

london_grpby_bor_period = london_clean.groupby(["survey_year","borough","period"]).agg("sum").reset_index()
london_grpby_bor_period = london_grpby_bor_weather[["survey_year",
                                                    "borough",
                                                    "period",
                                                    "total_cycles"]]

london_grpby_bor_period_mrg = pd.merge(london_grpby_bor_period,london_clean_cycling_area,on="borough",how="left")

london_clean_catplot_period = sns.catplot(
                            x="survey_year",
                            y= "total_cycles",
                            row= "borough",
                            col= "period",
                            kind="point",
                            hue="functional_cycling_area",
                            data=london_grpby_bor_weather_mrg) 

In [ ]:
london_clean_cycling_area = london_clean[["functional_cycling_area","borough"]]

london_grpby_bor_weather = london_clean.groupby(["survey_year","borough","season"]).agg("sum").reset_index()
london_grpby_bor_weather = london_grpby_bor_weather[["survey_year",
                                                    "borough",
                                                    "period",
                                                    "total_cycles"]]
london_grpby_bor_weather_mrg = pd.merge(london_grpby_bor_weather,london_clean_cycling_area,on="borough",how="left")

london_clean_catplot_mar = sns.catplot(
                            x="survey_year",
                            y= "total_cycles",
                            row= "borough",
                            col= "period",
                            kind="point",
                            hue="functional_cycling_area",
                            data=london_grpby_bor_weather_mrg) 

In [ ]:
london_clean_cycling_area = london_clean[["functional_cycling_area","borough"]]

london_grpby_bor_weather = london_clean.groupby(["survey_year","borough","day_of_week"]).agg("sum").reset_index()
london_grpby_bor_weather = london_grpby_bor_weather[["survey_year",
                                                    "borough",
                                                    "period",
                                                    "total_cycles"]]
london_grpby_bor_weather_mrg = pd.merge(london_grpby_bor_weather,london_clean_cycling_area,on="borough",how="left")

london_clean_catplot_mar = sns.catplot(
                            x="survey_year",
                            y= "total_cycles",
                            row= "borough",
                            col= "period",
                            kind="point",
                            hue="functional_cycling_area",
                            data=london_grpby_bor_weather_mrg) 

In [ ]:
sns.catplot(kind="box")

# Observations and Insights: Summary

# Testing for variable influence using Multilinear Regression

# Understanding demographic data

In [ ]:
london_clean.head()

# Male-female gender cyclist split

In [ ]:
london_clean_m_f_grpby = london_clean.groupby(["survey_year"]).agg("sum")
london_clean_m_f_grpby = london_clean_m_f_grpby.reset_index()
london_clean_m_f_grpby_subset = london_clean_m_f_grpby[["survey_year",
                                                        "number_of_male_cycles",
                                                        "number_of_female_cycles"]]
london_clean_m_f_grpby_subset = london_clean_m_f_grpby_subset.\
                                rename(columns={"number_of_male_cycles": "Male",
                                               "number_of_female_cycles":"Female"})

london_clean_m_f_grpby_subset = london_clean_m_f_grpby_subset.melt(id_vars=["survey_year"],
                                  var_name="Gender",
                                  value_name="Count")

fig, ax = (15,15)
sns.barplot(data=london_clean_m_f_grpby_subset,
           x="survey_year",
           y="Count",
           hue="Gender",
           palette="colorblind")\
.set_title("Cyclist Distribution by Gender")

# https://stackoverflow.com/questions/44548721/remove-row-with-null-value-from-pandas-data-frame

[11:52, 12/10/2022] Kevin Wijaya Oey: Females are significantly underrepresented compared to males on a year-on-year basis. [11:52, 12/10/2022] Kevin Wijaya Oey: Will require a t-test (if possible) to determine if the difference is significant. [11:54, 12/10/2022] Kevin Wijaya Oey: Questions that arise:

why are females underrepresented? Is it a lack of awareness or the fact that they don't have time to cycle due to other commitments (eg stay at home mums etc)?
demogrpahic data is only limited to certain areas, data is underrepresentative
greater decrease in males than females when COVID hit in 2020 [11:54, 12/10/2022] Javier Conde: Thanks Kevin [11:55, 12/10/2022] Kevin Wijaya Oey: - what caused the significant increase in female cyclist uptake between 2020 and 2021? Looks higher than pre-covid [11:55, 12/10/2022] Kevin Wijaya Oey: likewise with males, indicating perhaps people wanted to cycle during covid but couldnt

## Utilising a t-test to validate significance in gender distribution

# Private vs bikes for-hire 

In [ ]:
london_clean_prv_hire = london_clean2.groupby(["survey_year"]).agg("sum")
london_clean_prv_hire = london_clean_prv_hire.reset_index()
london_clean_prv_hire = london_clean_prv_hire[["survey_year", 
                                               "number_of_private_cycles",
                                               "number_of_cycle_hire_bikes"]]

london_clean_prv_hire = london_clean_prv_hire.rename(columns={"number_of_private_cycles": "Private",
                                               "number_of_cycle_hire_bikes":"Hire"})

london_clean_prv_hire = london_clean_prv_hire.melt(id_vars=["survey_year"],
                                  var_name="Bike_type",
                                  value_name="Count")

sns.barplot(data=london_clean_prv_hire,
           x="survey_year",
           y="Count",
           hue="Bike_type",
           palette=colorblind)\
.set_title("Cyclist Distribution by Bike_type")

# Utilising t-test to validate differences in private vs for-hire bikes

In [ ]:
london_clean2_grpby = london_clean2.groupby(["Survey_wave_year",
                       "Functional_cycling_area",
                      "season"]).agg("sum")
london_clean2_grpby = london_clean2_grpby[[]].reset_index()

sns.catplot(x="Survey_wave_year",
            y="Total_cycles",
           row="Functional_cycling_area",
           col="season",
           kind="point",
           data=london_clean2_grpby)

#bicycle_merge2_catplot_mar = sns.catplot(
#                            x="SiteID",
#                            y= "Count",
#                            row= "Year",
#                            col= "Time",
#                            kind="point",
#                            hue="Month",
#                            data=bicycle_merge2[bicycle_merge2["Month"]=="March"]) 


In [ ]:
#list(london_clean2)
#london_clean_m_f = london_clean2.pop("Number_of_private_cycles")
#list(london_clean2)
#london_clean2.pop("Number_of_cycle_hire_bikes")
#london_clean2.dropna(how="any",axis=0)
#list(london_clean2)

london_clean2.plot(kind="bar",
                  stacked=True,
                  color="red","blue")

#sns.barplot(data=london_clean2,
#           x="Survey_wave_year",
#           y=[["Number_of_male_cycles","Number_of_female_cycles"])



In [ ]:
fig = px.density_mapbox(london_clean2[london_clean2["Weather"]=="Good"],
                        lat='latitude', 
                        lon='longitude', 
                        z='TotalCycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="Survey_wave_year",
                        zoom=0,
                        hover_name="Functional Cycling Area",
                        hover_data={'Unnamed: 0':False,
                                     'Survey_wave_year':True,
                                     'Site_ID':False,
                                     'Location_x':True,
                                     'Survey_date':False,
                                     'Weather':True,
                                     'Time':True,
                                     'Period':True,
                                     'Direction':True,
                                     'Number_of_male_cycles':True,
                                     'Number_of_female_cycles':True,
                                     'Number_of_unknown_cycles':True,
                                     'Total_cycles':True,
                                     'Day_of_week':True,
                                     'season':True,
                                     'Number_of_private_cycles':True,
                                     'Number_of_cycle_hire_bikes':True,
                                     'SurveyDescription':True,
                                     'latitude':False,
                                     'longitude':False,
                                     'Location_y':False,
                                     'Borough':True,
                                     'Functional_cycling_area':True},
                        labels={"0600-0700":"6am-7am",
                               "0700-0800":"7am-8am",
                               "0800-0900":"8am-9am",
                               "1600-1700":"4pm-5pm",
                               "1700-1800":"5pm-6pm",
                               "1800-1900":"6pm-7pm"},
                        color_continuous_midpoint=True,
                        animation_group="Intersection",
                        title="Year-On-Year variation in total Cyclist Counts in Sydney for March",
                        height=1000,
                        width=1000,
                        opacity=1,
                        category_orders={"Year":["First"],
                                        "Month":["Second"],
                                        "TotalCount":["Third"],
                                        "0600-0700":["Fourth"],
                                        "0700-0800":["Fifth"],
                                        "0800-0900":["Sixth"],
                                        "1600-1700":["Seventh"],
                                        "1700-1800":["Eighth"],
                                        "1800-1900":["Ninth"]},
                        mapbox_style="stamen-terrain")
fig.show()

Business questions 
When approaching Thoughtworks, the Mayor of London proposed some initial questions:

- How can we increase the uptake of cycling in London?
- What are the main factors that determine whether people choose to cycle?
- What interventions and changes to the transport network have had the most impact on cycling engagement? 

Other questions to consider:

- What are the demographics of cyclists in these cities? 
- Are there any underrepresented groups that can be engaged with to try to increase the uptake of cycling as a mode of transport?

# New York Data

In [ ]:
# Load the ny data as a DataFrame.
ny_counts = pd.read_csv("ny_counts_Saurav_071022.csv")

# View the NY dataframe.
ny_counts.head() 

In [ ]:
# Remove the first column.
ny_counts.pop("Unnamed: 0")

In [ ]:
# List out the column names.
list(ny_counts)

In [ ]:
# Upload the geospatial data for bicycle counters in NY.
NYC_lat_long = pd.read_csv("NYC_Bicycle_Counters_Javier_08102022.csv")

# View the dataframe.
NYC_lat_long

In [ ]:
# Merge the ny_counts and location data.
NYC_clean_complete = pd.merge(ny_counts,NYC_lat_long,on="id",how="left")

# View the resultant dataframe.
NYC_clean_complete

# Remove unnecessary columns.
NYC_clean_complete.pop("domain")
NYC_clean_complete.pop("interval")
NYC_clean_complete.pop("timezone")
NYC_clean_complete.pop("sens")

# View the final NYC_clean
NYC_clean_complete

In [ ]:
NYC_lat_long_clean = NYC_lat_long[["id","name","latitude","longitude","interval","counter"]]
NYC_lat_long_clean.head()

In [ ]:
# Create an aggregated table to visualise the data with.
NYC_clean_agg_season = ny_counts.groupby(["year","month","season","id"]).agg("sum").reset_index( )

#View the resutlant dataframe.
NYC_clean_agg_season.head()

# Merge the aggregated table with location data.
NYC_season_final = pd.merge(NYC_clean_agg_season,NYC_lat_long_clean,on="id",how="left")
NYC_season_final2 = NYC_season_final[["year","month","season","counts"]]

# View the resultant dataframe.
NYC_season_final2

In [ ]:
# Create a facetplot to show the aggregated data.
# By season.
ny_catplot_1 = sns.catplot(x= "month", 
                           y="counts",
                           row="year", 
                           hue="season",
                           kind="point",
                          data=NYC_season_final2,
                          ci=False)

In [ ]:
# Create an aggregated table to visualise the data with.
NYC_clean_agg_time_of_day = ny_counts.groupby(["year","month","time_of_day"]).agg("sum").reset_index( )

#View the resutlant dataframe.
NYC_clean_agg_time_of_day.head()

# Merge the aggregated table with location data.
NYC_TOD_final = pd.merge(NYC_clean_agg_time_of_day,NYC_lat_long_clean,on="id",how="left")
NYC_TOD_final2 = NYC_TOD_final[["year","month","time_of_day","counts"]]

# View the resultant dataframe.
NYC_TOD_final2

# By time period.
ny_catplot_2 = sns.catplot(data=NYC_TOD_final2, 
                           x="time_of_day", 
                           y="counts",
                           row="year", 
                           col="month", 
                           kind="point")

In [ ]:
# Create an aggregated table to visualise the data with.
NYC_clean_agg_DOW = ny_counts.groupby(["year","month","day_of_week"]).agg("sum").reset_index( )

#View the resutlant dataframe.
NYC_clean_agg_DOW.head()

# Merge the aggregated table with location data.
NYC_DOW_final = pd.merge(NYC_clean_agg_DOW,NYC_lat_long_clean,on="id",how="left")
NYC_DOW_final2 = NYC_DOW_final[["year","month","day_of_week","counts"]]

# View the resultant dataframe.
NYC_DOW_final2

# By time period.
ny_catplot_3 = sns.catplot(data=NYC_DOW_final2, 
                           x="day_of_week", 
                           y="counts",
                           row="year", 
                           col="month", 
                           kind="point")

In [ ]:
# Macro view 
sns.lineplot(data=ny_counts, x=month_year, y= total_counts)

# Predictive Analytics
# Create a multilinear regression model showcasing how timing affects things.
# Wait wouldnt they be collinear...and htat would stuff up the model. sigh

In [ ]:
# use t-tests, mlr, and scenaroi analysis

In [ ]:
# Prepare NY data - aggregated by season.
ny_season_agg = ny_counts.groupby(["id","year","season"]).agg("sum").reset_index().sort_values(by="year",ascending=True)
ny_season_agg2 = ny_season_agg[["id","year","month","season","counts"]]
NYC_season_agg_mrg = pd.merge(ny_season_agg2,NYC_lat_long,on="id",how="left")
NYC_season_agg_mrg2 = NYC_season_agg_mrg[["year","season","counts","name","latitude","longitude"]]

# Prepare NY data - aggregated by day_of_week.
ny_DOW_agg = ny_counts.groupby(["id","year","day_of_week"]).agg("sum").reset_index().sort_values(by="year",ascending=True)
ny_DOW_agg2 = ny_DOW_agg[["id","year","day_of_week","counts"]]
NYC_DOW_agg_mrg = pd.merge(ny_DOW_agg2,NYC_lat_long,on="id",how="left")
NYC_DOW_agg_mrg2 = NYC_DOW_agg_mrg[["year","day_of_week","counts","name","latitude","longitude"]]

# Prepare NY data - aggregated by time_of_day.
ny_TOD_agg = ny_counts.groupby(["id","year","date","time_of_day"]).agg("sum").reset_index().sort_values(by=["year","date","time_of_day"],ascending=True)
ny_TOD_agg2 = ny_TOD_agg[["id","year","time_of_day","counts"]]
NYC_TOD_agg_mrg = pd.merge(ny_TOD_agg2,NYC_lat_long,on="id",how="left")
NYC_TOD_agg_mrg2 = NYC_TOD_agg_mrg[["year","time_of_day","counts","name","latitude","longitude"]]

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by season.
fig = px.density_mapbox(NYC_season_agg_mrg2[NYC_season_agg_mrg2["season"]=="Summer"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "season": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York in Winter",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by season.
fig = px.density_mapbox(NYC_season_agg_mrg2[NYC_season_agg_mrg2["season"]=="Winter"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "season": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York in Winter",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by season.
fig = px.density_mapbox(NYC_season_agg_mrg2[NYC_season_agg_mrg2["season"]=="Spring"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "season": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York in Spring",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by season.
fig = px.density_mapbox(NYC_season_agg_mrg2[NYC_season_agg_mrg2["season"]=="Autumn"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "season": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York in Autumn",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
pd.unique(ny_counts["day_of_week"])

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by day of week.
fig = px.density_mapbox(NYC_DOW_agg_mrg2[NYC_DOW_agg_mrg2["day_of_week"]=="Monday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "day_of_week": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York on Mondays",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by day of week.
fig = px.density_mapbox(NYC_DOW_agg_mrg2[NYC_DOW_agg_mrg2["day_of_week"]=="Tuesday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "day_of_week": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York on Mondays",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by day of week.
fig = px.density_mapbox(NYC_DOW_agg_mrg2[NYC_DOW_agg_mrg2["day_of_week"]=="Wednesday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "day_of_week": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York on Mondays",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by day of week.
fig = px.density_mapbox(NYC_DOW_agg_mrg2[NYC_DOW_agg_mrg2["day_of_week"]=="Thursday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "day_of_week": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York on Mondays",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by day of week.
fig = px.density_mapbox(NYC_DOW_agg_mrg2[NYC_DOW_agg_mrg2["day_of_week"]=="Friday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "day_of_week": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York on Mondays",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by day of week.
fig = px.density_mapbox(NYC_DOW_agg_mrg2[NYC_DOW_agg_mrg2["day_of_week"]=="Saturday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "day_of_week": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York on Mondays",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by day of week.
fig = px.density_mapbox(NYC_DOW_agg_mrg2[NYC_DOW_agg_mrg2["day_of_week"]=="Sunday"],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "day_of_week": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York on Mondays",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
ny_counts.sort_values(by="date",ascending=True)
#pd.unique(ny_counts["time_of_day","time"])
#NYC_TOD_agg_mrg2

In [ ]:
# Prepare heatmap for NY based on YoY changes in total count by day of week.
fig = px.density_mapbox(NYC_TOD_agg_mrg2[NYC_TOD_agg_mrg2["time_of_day"]==""],
                        lat='latitude', 
                        lon='longitude', 
                        z='counts', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="name",
                        hover_data={"year": True,
                                   "day_of_week": True,
                                   "counts": True,
                                   "name": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York on Mondays",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")
fig.show()

# Impact of Covid on cycling

In [ ]:
london_covid = pd.read_csv("london_plus_covid_vanessa_081022.csv")
london_covid.head()
print(pd.unique(london_covid["stay_home_requirements"]))

In [ ]:
london_covid = pd.read_csv("london_plus_covid_vanessa_081022.csv")

london_covid_grpby = london_covid.groupby(["survey_year","covid_status","site_id"]).agg("sum").reset_index()
london_covid_grpby = london_covid_grpby[["survey_year","covid_status","site_id","total_cycles"]]
london_covid_grpby_mrg = pd.merge(london_covid_grpby, london_locations,on="site_id",how="left")

fig = px.density_mapbox(london_covid_grpby_,rg,
                        lat='latitude', 
                        lon='longitude', 
                        z='TotalCycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="covid_status",
                        zoom=0,
                        hover_name="Functional Cycling Area",
                        hover_data={'Unnamed: 0':False,
                                     'Survey_wave_year':True,
                                     'Site_ID':False,
                                     'Location_x':True,
                                     'Survey_date':False,
                                     'Weather':True,
                                     'Time':True,
                                     'Period':True,
                                     'Direction':True,
                                     'Number_of_male_cycles':True,
                                     'Number_of_female_cycles':True,
                                     'Number_of_unknown_cycles':True,
                                     'Total_cycles':True,
                                     'Day_of_week':True,
                                     'season':True,
                                     'Number_of_private_cycles':True,
                                     'Number_of_cycle_hire_bikes':True,
                                     'SurveyDescription':True,
                                     'latitude':False,
                                     'longitude':False,
                                     'Location_y':False,
                                     'Borough':True,
                                     'Functional_cycling_area':True},
                        labels={"0600-0700":"6am-7am",
                               "0700-0800":"7am-8am",
                               "0800-0900":"8am-9am",
                               "1600-1700":"4pm-5pm",
                               "1700-1800":"5pm-6pm",
                               "1800-1900":"6pm-7pm"},
                        color_continuous_midpoint=True,
                        animation_group="Intersection",
                        title="Year-On-Year variation in total Cyclist Counts in Sydney for March",
                        height=1000,
                        width=1000,
                        opacity=1,
                        category_orders={"Year":["First"],
                                        "Month":["Second"],
                                        "TotalCount":["Third"],
                                        "0600-0700":["Fourth"],
                                        "0700-0800":["Fifth"],
                                        "0800-0900":["Sixth"],
                                        "1600-1700":["Seventh"],
                                        "1700-1800":["Eighth"],
                                        "1800-1900":["Ninth"]},
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
!pip install gis

In [ ]:
!pip install arcgis

In [ ]:
from arcgis.gis import GIS
my_gis = GIS()
#m = my_gis.map()
#m.add_layer("london_plus_covid_vanessa_081022.csv")

m = my_gis.map("London")
m.add_layer()

# Making heatmaps with predictive models 

In [ ]:
london_clean.duplicated().sum()

In [ ]:
london_mrg = london_clean.merge(london_locations,on="site_id",how="left")
london_mrg.duplicated().sum()

In [ ]:
bike_site = pd.read_csv("bike_reconv.csv")
bike_site.rename(columns = {"UnqID": "site_id"},inplace=True)
a = london_clean.merge(bike_site,on="site_id",how="left")
a.duplicated().sum()

In [ ]:
# libraries in python to interactive >. kind of gets you there?
# plotly filtering >> for final 
#plotly > add filters? i'd like to digigng

In [ ]:
london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_period2 = london_clean.groupby(["site_id", "season","period","time"]).agg("sum").reset_index()
london_period2 = london_period2.sort_values(by="time",ascending=True)
london_period2 = london_period2[["site_id","season","period","total_cycles"]]

london_period2_data = pd.merge(london_period2,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_period2_data,
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="period",
                        zoom=0,
                        hover_name="location",
                        hover_data={"site_id":False,
                                    "period":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False,
                                   "season":True},
                                      title="Average number of cyclists throughout different times of the day",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")

fig.update_layout(
    updatemenus=[
        dict(active=0,
            buttons=list([
            dict(label="None",
                 method="update",
                 args=[{"title":"ALL"}]),
            dict(label="Winter",
                 method="update",
                 args=[{"title":"Winter"}]),
            dict(label="Summer",
                 method="update",
                 args=[{"title":"Summer"}]),
            dict(label="Spring",
                 method="update",
                 args=[{"title":"Spring"}])
        ]),
        )
    ]
)

fig.show()



[11:52, 12/10/2022] Kevin Wijaya Oey: Females are significantly underrepresented compared to males on a year-on-year basis.
[11:52, 12/10/2022] Kevin Wijaya Oey: Will require a t-test (if possible) to determine if the difference is significant.
[11:54, 12/10/2022] Kevin Wijaya Oey: Questions that arise:
- why are females underrepresented? Is it a lack of awareness or the fact that they don't have time to cycle due to other commitments (eg stay at home mums etc)? 
- demogrpahic data is only limited to certain areas, data is underrepresentative
- greater decrease in males than females when COVID hit in 2020
[11:54, 12/10/2022] Javier Conde: Thanks Kevin
[11:55, 12/10/2022] Kevin Wijaya Oey: - what caused the significant increase in female cyclist uptake between 2020 and 2021? Looks higher than pre-covid
[11:55, 12/10/2022] Kevin Wijaya Oey: likewise with males, indicating perhaps people wanted to cycle during covid but couldnt

In [ ]:
## Try changing the graph and DF to london)_clean and the heatmap?

# Imports
import plotly.graph_objs as go
import pandas as pd
import numpy as np

london_locations = london_clean[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_period2 = london_clean.groupby(["site_id", "season","period","time"]).agg("sum").reset_index()
london_period2 = london_period2.sort_values(by="time",ascending=True)
london_period2 = london_period2[["site_id","season","period","total_cycles"]]

# source data
df = pd.merge(london_period2,london_locations,how="left",on="site_id")

df_input = df.copy()

# split df by labels
labels = df['site_id'].unique().tolist()
dates = df['season'].unique().tolist()

# dataframe collection grouped by labels
dfs = {}
for label in labels:
    dfs[label]=pd.pivot_table(df[df['site_id']==label],
                                    values='total_cycles',
                                    index=['season',"period"],
                                    columns=[],
                                    aggfunc=np.sum)

# find row and column unions
common_cols = []
common_rows = []
for df in dfs.keys():
    common_cols = sorted(list(set().union(common_cols,list(dfs[df]))))
    common_rows = sorted(list(set().union(common_rows,list(dfs[df].index))))

# find dimensionally common dataframe
df_common = pd.DataFrame(np.nan, index=common_rows, columns=common_cols)

# reshape each dfs[df] into common dimensions
dfc={}
for df_item in dfs:
    #print(dfs[unshaped])
    df1 = dfs[df_item].copy()
    s=df_common.combine_first(df1)
    df_reshaped = df1.reindex_like(s)
    dfc[df_item]=df_reshaped

# plotly start 

fig = px.density_mapbox(df,
                        lat='latitude', 
                        lon='longitude', 
                        z='total_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="period",
                        zoom=0,
                        hover_name="location",
                        hover_data={"site_id":False,
                                    "period":True,
                                    "total_cycles":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False,
                                   "season":True},
                                      title="Average number of cyclists throughout different times of the day",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")

# menu setup    
updatemenu= []

# buttons for menu 1, names
buttons=[]

# create traces for each color: 
# build argVals for buttons and create buttons
for df in dfc.keys():
    argList = []
    for col in dfc[df]:
        #print(dfc[df][col].values)
        argList.append(dfc[df][col].values)
    argVals = [ {'y':argList}]

    buttons.append(dict(method='update',
                        label=df,
                        visible=True,
                        args=argVals))

# buttons for menu 2, colors
b2_labels = common_cols

# matrix to feed all visible arguments for all traces
# so that they can be shown or hidden by choice
b2_show = [list(b) for b in [e==1 for e in np.eye(len(b2_labels))]]
buttons2=[]
buttons2.append({'method': 'update',
                 'label': 'All',
                 'args': [{'visible': [True]*len(common_cols)}]})

# create buttons to show or hide
for i in range(0, len(b2_labels)):
    buttons2.append(dict(method='update',
                        label=b2_labels[i],
                        args=[{'visible':b2_show[i]}]
                        )
                   )

# add option for button two to hide all
buttons2.append(dict(method='update',
                        label='None',
                        args=[{'visible':[False]*len(common_cols)}]
                        )
                   )

# some adjustments to the updatemenus
updatemenu=[]
your_menu=dict()
updatemenu.append(your_menu)
your_menu2=dict()
updatemenu.append(your_menu2)
updatemenu[1]
updatemenu[0]['buttons']=buttons
updatemenu[0]['direction']='down'
updatemenu[0]['showactive']=True
updatemenu[1]['buttons']=buttons2
updatemenu[1]['y']=0.6

fig.update_layout(showlegend=False, updatemenus=updatemenu)
fig.update_layout(yaxis=dict(range=[0,df_input['value'].max()+0.4]))

# title
fig.update_layout(
    title=dict(
        text= "<i>Filtering with multiple dropdown buttons</i>",
        font={'size':18},
        y=0.9,
        x=0.5,
        xanchor= 'center',
        yanchor= 'top'))

# button annotations
fig.update_layout(
    annotations=[
        dict(text="<i>Label</i>", x=-0.2, xref="paper", y=1.1, yref="paper",
            align="left", showarrow=False, font = dict(size=16, color = 'steelblue')),
        dict(text="<i>Color</i>", x=-0.2, xref="paper", y=0.7, yref="paper",
            align="left", showarrow=False, font = dict(size=16, color = 'steelblue')

                             )
    ])

fig.show()

In [ ]:

# Imports
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# source data
df = pd.DataFrame({0: {'num': 1, 'label': 'A', 'color': 'red', 'value': 0.4},
                    1: {'num': 2, 'label': 'A', 'color': 'blue', 'value': 0.2},
                    2: {'num': 3, 'label': 'A', 'color': 'green', 'value': 0.3},
                    3: {'num': 4, 'label': 'A', 'color': 'red', 'value': 0.6},
                    4: {'num': 5, 'label': 'A', 'color': 'blue', 'value': 0.7},
                    5: {'num': 6, 'label': 'A', 'color': 'green', 'value': 0.4},
                    6: {'num': 7, 'label': 'B', 'color': 'blue', 'value': 0.2},
                    7: {'num': 8, 'label': 'B', 'color': 'green', 'value': 0.4},
                    8: {'num': 9, 'label': 'B', 'color': 'red', 'value': 0.4},
                    9: {'num': 10, 'label': 'B', 'color': 'green', 'value': 0.2},
                    10: {'num': 11, 'label': 'C', 'color': 'red', 'value': 0.1},
                    11: {'num': 12, 'label': 'C', 'color': 'blue', 'value': 0.3},
                    12: {'num': 13, 'label': 'D', 'color': 'red', 'value': 0.8},
                    13: {'num': 14, 'label': 'D', 'color': 'blue', 'value': 0.4},
                    14: {'num': 15, 'label': 'D', 'color': 'green', 'value': 0.6},
                    15: {'num': 16, 'label': 'D', 'color': 'yellow', 'value': 0.5},
                    16: {'num': 17, 'label': 'E', 'color': 'purple', 'value': 0.68}}
                    ).T

df_input = df.copy()

# split df by labels
labels = df['label'].unique().tolist()
dates = df['num'].unique().tolist()


# dataframe collection grouped by labels
dfs = {}
for label in labels:
    dfs[label]=pd.pivot_table(df[df['label']==label],
                                    values='value',
                                    index=['num'],
                                    columns=['color'],
                                    aggfunc=np.sum)

# find row and column unions
common_cols = []
common_rows = []
for df in dfs.keys():
    common_cols = sorted(list(set().union(common_cols,list(dfs[df]))))
    common_rows = sorted(list(set().union(common_rows,list(dfs[df].index))))


# find dimensionally common dataframe
df_common = pd.DataFrame(np.nan, index=common_rows, columns=common_cols)

# reshape each dfs[df] into common dimensions
dfc={}
for df_item in dfs:
    #print(dfs[unshaped])
    df1 = dfs[df_item].copy()
    s=df_common.combine_first(df1)
    df_reshaped = df1.reindex_like(s)
    dfc[df_item]=df_reshaped

# plotly start 
fig = go.Figure()
# one trace for each column per dataframe: AI and RANDOM
for col in common_cols:
    fig.add_trace(go.Scatter(x=dates,
                             visible=True,
                             marker=dict(size=12, line=dict(width=2)),
                             marker_symbol = 'diamond',name=col
                  )
             )

# menu setup    
updatemenu= []

# buttons for menu 1, names
buttons=[]

# create traces for each color: 
# build argVals for buttons and create buttons
for df in dfc.keys():
    argList = []
    for col in dfc[df]:
        #print(dfc[df][col].values)
        argList.append(dfc[df][col].values)
    argVals = [ {'y':argList}]

    buttons.append(dict(method='update',
                        label=df,
                        visible=True,
                        args=argVals))

# buttons for menu 2, colors
b2_labels = common_cols

# matrix to feed all visible arguments for all traces
# so that they can be shown or hidden by choice
b2_show = [list(b) for b in [e==1 for e in np.eye(len(b2_labels))]]
buttons2=[]
buttons2.append({'method': 'update',
                 'label': 'All',
                 'args': [{'visible': [True]*len(common_cols)}]})

# create buttons to show or hide
for i in range(0, len(b2_labels)):
    buttons2.append(dict(method='update',
                        label=b2_labels[i],
                        args=[{'visible':b2_show[i]}]
                        )
                   )

# add option for button two to hide all
buttons2.append(dict(method='update',
                        label='None',
                        args=[{'visible':[False]*len(common_cols)}]
                        )
                   )

# some adjustments to the updatemenus
updatemenu=[]
your_menu=dict()
updatemenu.append(your_menu)
your_menu2=dict()
updatemenu.append(your_menu2)
updatemenu[1]
updatemenu[0]['buttons']=buttons
updatemenu[0]['direction']='down'
updatemenu[0]['showactive']=True
updatemenu[1]['buttons']=buttons2
updatemenu[1]['y']=0.6

fig.update_layout(showlegend=False, updatemenus=updatemenu)
fig.update_layout(yaxis=dict(range=[0,df_input['value'].max()+0.4]))

# title
fig.update_layout(
    title=dict(
        text= "<i>Filtering with multiple dropdown buttons</i>",
        font={'size':18},
        y=0.9,
        x=0.5,
        xanchor= 'center',
        yanchor= 'top'))

# button annotations
fig.update_layout(
    annotations=[
        dict(text="<i>Label</i>", x=-0.2, xref="paper", y=1.1, yref="paper",
            align="left", showarrow=False, font = dict(size=16, color = 'steelblue')),
        dict(text="<i>Color</i>", x=-0.2, xref="paper", y=0.7, yref="paper",
            align="left", showarrow=False, font = dict(size=16, color = 'steelblue')

                             )
    ])

fig.show()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

import plotly.express as px

fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
counties["features"][0]

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/radoi90/housequest-data/master/london_boroughs.geojson') as response:
    counties = json.load(response)

#import pandas as pd
#df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
#                   dtype={"fips": str})

import plotly.express as px

fig = px.choropleth_mapbox(london_clean, geojson=counties, locations='latitude', color='functional_cycling_area',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import pandas as pd
#traff_boro = pd.read_csv("traffic_flow_borough.csv")
#traff_boro.melt(id_vars="Year",var_name="Borough",value_name="Count")
priv_cars_LDN = pd.read_csv("private_cars_london.csv")
priv_vehcle_boro_LDN = pd.read_csv("private_vehicles_by_borough_london.csv")

london_final = pd.read_csv("london_count_and_site_Saurav_161022.csv")
#list(london_final)

In [ ]:
priv_cars_LDN_new = priv_cars_LDN.melt(id_vars="Year",var_name="Borough",value_name="Count")

In [ ]:
priv_vehcle_boro_LDN = pd.read_csv("private_vehicles_by_borough_london.csv")
priv_vehcle_boro_LDN_new = priv_vehcle_boro_LDN.melt(id_vars="Borough",var_name="Year",value_name="Count")
priv_vehcle_boro_LDN_new
pd.unique(priv_vehcle_boro_LDN_new["Borough"])

In [ ]:
london_final = london_final.rename(columns={"borough":"Borough"})
london_final2 = pd.merge(london_final,priv_vehcle_boro_LDN_new,on="Borough",how="left")
list(london_final2)

In [ ]:
priv_vehcle_boro_LDN_new["Year"] = priv_vehcle_boro_LDN_new["Year"].astype("int")
priv_vehcle_boro_LDN_new["Count"] = priv_vehcle_boro_LDN_new["Count"].astype("int")
priv_vehcle_boro_LDN_new.dtypes
priv_vehcle_boro_LDN_new.to_csv("priv_vehcle_boro_LDN_new.csv")
priv_vehcle_boro_LDN_new[priv_vehcle_boro_LDN_new["Year"] >=2010]
#priv_vehcle_boro_LDN_new["boro_yr"] = priv_vehcle_boro_LDN_new["Borough"] + "_" + priv_vehcle_boro_LDN_new["Year"]
#priv_vehcle_boro_LDN_new.query("Year>='2021'")
#priv_vehcle_boro_LDN_new = priv_vehcle_boro_LDN_new["Year"].astype("float")
#priv_vehcle_boro_LDN_new.head()

In [ ]:
priv_vehcle_boro_LDN_new2 = priv_vehcle_boro_LDN_new[priv_vehcle_boro_LDN_new["Year"] >=2014]
priv_vehcle_boro_LDN_new2["boro_yr"] = priv_vehcle_boro_LDN_new2["Borough"] +"_" + priv_vehcle_boro_LDN_new2["Year"].astype("string")
priv_vehcle_boro_LDN_new2.head(3)

In [ ]:
priv_cars_LDN = pd.read_csv("private_cars_london.csv")
priv_vehcle_boro_LDN = pd.read_csv("private_vehicles_by_borough_london.csv")

priv_cars_LDN_new.sort_values(by="Year",ascending=True)

In [ ]:
london_final["boro_yr"] = london_final["borough"] + "_" + london_final["survey_year"].astype("string")
london_final.head(3)
london_final["survey_year"].min()
london_final["survey_year"].max()

In [ ]:
london_final2 = pd.merge(london_final,priv_vehcle_boro_LDN_new2,on="boro_yr",how="left")
london_final2.rename(columns={"Count":"Total Traffic"})



# repeat for other datasets
# and load into python or plotly
# using chloropleths
# do some research on actionable insishgts

In [ ]:
traff_boro = pd.read_csv("traffic_flow_borough.csv")
traff_boro2 = traff_boro.melt(id_vars="Year",var_name="Borough",value_name="Count")
traff_boro2["boro_yr"] = traff_boro2["Borough"] + "_" + traff_boro2["Year"].astype("string")
traff_boro2[["boro_yr","Count"]].rename(columns={"Count":"Total Traffic"})

In [ ]:
priv_cars_LDN = pd.read_csv("private_cars_london.csv")
priv_cars_LDN2 = priv_cars_LDN.melt(id_vars="Year",var_name="Borough",value_name="Count")
priv_cars_LDN2["boro_yr"] = priv_cars_LDN2["Borough"] + "_" + priv_cars_LDN2["Year"].astype("string")
priv_cars_LDN2[["boro_yr","Count"]].rename(columns={"Count":"Total Private Cars"})


In [ ]:
london_final3 = pd.merge(london_final,priv_vehcle_boro_LDN_new2,on="boro_yr",how="left")
london_final4 = pd.merge(london_final,priv_vehcle_boro_LDN_new2,on="boro_yr",how="left")

# add popn data too 
# n sleep 
#for now
# fnsh crossove rn reapply for jobs
tmr
pls

# add buttons for px.mapbox

In [ ]:
fig1 = px.scatter_mapbox(london_final, 
                         lat="latitude", 
                         lon="longitude", 
                         hover_name="borough", 
                         hover_data=["borough"],
                        color_discrete_sequence=["fuchsia"], 
                         zoom=10, 
                         height=300)
fig1.show()

In [ ]:
fig2 = px.density_mapbox(london_final[london_final["survey_year"]==2015],
                        lat='latitude', 
                        lon='longitude', 
                        z='number_of_male_cycles', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="borough",
                        hover_data={"year": True,
                                   "day_of_week": True,
                                   "borough": True,
                                   "latitude":False,
                                   "longitude":False},
                        labels={"name":"Counter Location",
                               "counts":"Number of cyclists"},
                        color_continuous_midpoint=True,
                        title="Year-On-Year variation in total Cyclist Counts in New York on Mondays",
                        height=1000,
                        width=1000,
                        opacity=1,
                        mapbox_style="stamen-terrain")

#trace0 = fig2 # the second map from the previous code
#fig1.add_trace(trace0.data[0])
#trace0.layout.update(showlegend=False)
fig2.show()

In [ ]:
list(london_clean)

In [ ]:
outer_london2 = pd.read_csv("Outer London.csv")
outer_london2
#london_loc2 = london_locations.copy().rename(columns={"Site_ID":"Site ID"})
#pd.merge(outer_london2,london_loc2,on="Site ID",how="left")